In [ ]:
############# MF CF IMPLEMMENTATION FROM SCRATCH ##############

# Simulated user-item rating matrix
# Rows: users
# Columns: items
# 0 indicates missing values
R_df = pd.read_parquet("dataset/adj_matrix_contract_names.parquet")
R = R_df.values

# Initialize parameters
num_users, num_items = R.shape
num_features = 3  # Number of latent features
learning_rate = 0.01  # Learning rate
regularization = 0.01  # Regularization parameter
num_epochs = 100  # Number of epochs

# Initialize user and item matrices with random values
U = np.random.randn(num_users, num_features)
I = np.random.randn(num_items, num_features)

print('1')
# Training
for epoch in tqdm(range(num_epochs), total = num_epochs):
    for i in range(num_users):
        for j in range(num_items):
            if R[i][j] > 0:
                # Compute error
                eij = R[i][j] - np.dot(U[i, :], I[j, :].T)
                
                # Update matrices
                for k in range(num_features):
                    U[i][k] += learning_rate * (2 * eij * I[j][k] - regularization * U[i][k])
                    I[j][k] += learning_rate * (2 * eij * U[i][k] - regularization * I[j][k])
    
    print('2')
    # Compute total error
    error = 0
    for i in range(num_users):
        for j in range(num_items):
            if R[i][j] > 0:
                error += (R[i][j] - np.dot(U[i, :], I[j, :].T)) ** 2
                for k in range(num_features):
                    error += (regularization / 2) * (U[i][k] ** 2 + I[j][k] ** 2)
    print('3')
    # print(f"Epoch {epoch + 1}/{num_epochs} - Error: {error}")

# Create the prediction matrix
prediction = np.dot(U, I.T)

# print("Prediction matrix:")
# print(prediction)

def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k]
    return np.mean(r)

def average_precision(r, k):
    r = np.asarray(r)[:k]
    out = [precision_at_k(r, j + 1) for j, rel in enumerate(r) if rel]
    if not out:
        return 0
    return np.mean(out)

def mean_average_precision(rs, k):
    return np.mean([average_precision(r, k) for r in rs])

# Assume these are your predicted ratings
predicted_ratings = prediction

# Assume these are your true binary relevance scores (e.g., 1 if relevant, 0 otherwise)
true_relevance = pd.read_parquet("dataset/adj_matrix_contract_names.parquet")

# Convert predicted ratings to binary relevance scores based on a threshold (optional)
predicted_relevance = (predicted_ratings > 0.5).astype(int)

# Sort the true relevance labels based on predicted ratings (higher ratings are assumed to be more relevant)
sorted_relevance = []
for i in range(predicted_ratings.shape[0]):
    sorted_relevance.append(true_relevance[i, np.argsort(predicted_ratings[i])[::-1]])

# Calculate MAP@K
K = 5
result = mean_average_precision(sorted_relevance, K)
print(f"MAP@{K}: {result}")

In [ ]:
################# PRECISION & RECALL EVAL ######################
def precision_recall_at_k(predictions, k, threshold):
    '''Return precision and recall at k metrics for each user.'''
    precisions = dict()
    recalls = dict()
    user_est_true = defaultdict(list)

    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings) 
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k]) 
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0 

    return precisions, recalls, k